# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [18]:
# dt=df.dtypes
# dummy_cols=dt[dt != 'float64'].index.values
# df_dummies_encoded = pd.get_dummies(df[dummy_cols].drop(columns=target))
df_dummies_encoded.value_counts()

home_ownership_ANY  home_ownership_MORTGAGE  home_ownership_OWN  home_ownership_RENT  verification_status_Not Verified  verification_status_Source Verified  verification_status_Verified  issue_d_Feb-2019  issue_d_Jan-2019  issue_d_Mar-2019  pymnt_plan_n  initial_list_status_f  initial_list_status_w  next_pymnt_d_Apr-2019  next_pymnt_d_May-2019  application_type_Individual  application_type_Joint App  hardship_flag_N  debt_settlement_flag_N
0                   1                        0                   0                    1                                 0                                    0                             0                 1                 0                 1             0                      1                      0                      1                      1                            0                           1                1                         3179
                                                                                      0                 

# Split the Data into Training and Testing

In [26]:
# Create our features
#can just apply get_dummies to df, since all object/categorical data is...
#automatically converted to dummy vars by the func
df_encoded = pd.get_dummies(df[columns].drop(columns=target))
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
df_scaled = data_scaler.fit_transform(df_encoded)
#X = pd.get_dummies(df[columns]) # YOUR CODE HERE
X = df_scaled.copy()
X=pd.DataFrame(X,columns=df_encoded.columns)
# X =  df_encoded
# # Create our target

y = df[target]# YOUR CODE HERE


In [27]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,...,6.881700e+04,68817.0,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,68817.0,68817.0
mean,-7.361679e-18,-1.199557e-15,6.590079e-17,6.016378e-16,-9.826434e-17,-2.177913e-15,-1.431710e-15,1.546511e-16,-4.943241e-15,-8.168287e-17,...,1.720205e-13,0.0,-8.229273e-15,8.228426e-15,-5.500409e-14,5.500409e-14,1.836062e-14,-1.734938e-14,0.0,0.0
std,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,...,1.000007e+00,0.0,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,0.0,0.0
min,-1.525462e+00,-1.407007e+00,-1.561349e+00,-7.628861e-01,-1.078175e+00,-3.031425e-01,-6.564915e-01,-1.757869e+00,-3.742043e-01,-8.062084e-01,...,-4.641321e-01,0.0,-3.760255e-01,-2.659394e+00,-7.881427e-01,-1.268806e+00,-2.481982e+00,-4.029039e-01,0.0,0.0
25%,-7.470458e-01,-8.231618e-01,-7.461036e-01,-3.306281e-01,-3.905201e-01,-3.031425e-01,-6.564915e-01,-7.616592e-01,-3.742043e-01,-5.180109e-01,...,-4.641321e-01,0.0,-3.760255e-01,3.760255e-01,-7.881427e-01,-1.268806e+00,4.029039e-01,-4.029039e-01,0.0,0.0
50%,-1.632334e-01,-2.019175e-01,-2.641560e-01,-1.316302e-01,-9.991302e-02,-3.031425e-01,-6.564915e-01,-2.635541e-01,-3.742043e-01,-2.535361e-01,...,-4.641321e-01,0.0,-3.760255e-01,3.760255e-01,-7.881427e-01,7.881427e-01,4.029039e-01,-4.029039e-01,0.0,0.0
75%,7.124852e-01,5.813905e-01,5.812919e-01,1.365843e-01,2.416864e-01,-3.031425e-01,6.625677e-01,5.666210e-01,-3.742043e-01,1.891789e-01,...,-4.641321e-01,0.0,-3.760255e-01,3.760255e-01,1.268806e+00,7.881427e-01,4.029039e-01,-4.029039e-01,0.0,0.0
max,2.269318e+00,3.754100e+00,4.150440e+00,7.535346e+01,4.837948e+01,2.475388e+01,5.938804e+00,9.864583e+00,1.150248e+01,2.608509e+01,...,2.154559e+00,0.0,2.659394e+00,3.760255e-01,1.268806e+00,7.881427e-01,4.029039e-01,2.481982e+00,0.0,0.0


In [28]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
print(np.unique(y_test, return_counts=True))
print(np.unique(y_train,return_counts=True))

(array(['high_risk', 'low_risk'], dtype=object), array([  101, 17104], dtype=int64))
(array(['high_risk', 'low_risk'], dtype=object), array([  246, 51366], dtype=int64))


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [30]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_train_ros, y_train_ros = ros.fit_resample(X_train,y_train)
Counter(y_train_ros)
y_train_ros.value_counts()

loan_status
high_risk      51366
low_risk       51366
dtype: int64

In [31]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs',random_state=1)
model.fit(X_train_ros,y_train_ros)

LogisticRegression(random_state=1)

In [32]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)



0.8325468421491353

In [33]:
# Display the confusion matrix
#y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   83,    18],
       [ 2680, 14424]], dtype=int64)

In [34]:
np.unique(y_train,return_counts=True)

(array(['high_risk', 'low_risk'], dtype=object),
 array([  246, 51366], dtype=int64))

In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.82      0.84      0.06      0.83      0.69       101
   low_risk       1.00      0.84      0.82      0.91      0.83      0.69     17104

avg / total       0.99      0.84      0.82      0.91      0.83      0.69     17205



### SMOTE Oversampling

In [37]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_train_smote, y_train_smote = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_train_smote)
y_train_smote.value_counts()

loan_status
high_risk      51366
low_risk       51366
dtype: int64

In [38]:
# Train the Logistic Regression model using the resampled data
model.fit(X_train_smote,y_train_smote)

LogisticRegression(random_state=1)

In [39]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.8440938486973113

In [40]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[   83,    18],
       [ 2285, 14819]], dtype=int64)

In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.82      0.87      0.07      0.84      0.71       101
   low_risk       1.00      0.87      0.82      0.93      0.84      0.72     17104

avg / total       0.99      0.87      0.82      0.92      0.84      0.72     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [42]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_train_cc, y_train_cc = cc.fit_resample(X_train, y_train)
Counter(y_train_cc)
y_train_cc.value_counts()

loan_status
high_risk      246
low_risk       246
dtype: int64

In [43]:
# Train the Logistic Regression model using the resampled data
model.fit(X_train_cc, y_train_cc)

LogisticRegression(random_state=1)

In [44]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8203882595930314

In [45]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[   89,    12],
       [ 4112, 12992]], dtype=int64)

In [46]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.88      0.76      0.04      0.82      0.68       101
   low_risk       1.00      0.76      0.88      0.86      0.82      0.66     17104

avg / total       0.99      0.76      0.88      0.86      0.82      0.66     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [47]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_train_smoteenn, y_train_smoteenn = smote_enn.fit_resample(X_train, y_train)
Counter(y_train_smoteenn)
y_train_smoteenn.value_counts()

loan_status
high_risk      51366
low_risk       47640
dtype: int64

In [48]:
# Train the Logistic Regression model using the resampled data
model.fit(X_train_smoteenn, y_train_smoteenn)

LogisticRegression(random_state=1)

In [49]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8388903875186395

In [50]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[   83,    18],
       [ 2463, 14641]], dtype=int64)

In [51]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.82      0.86      0.06      0.84      0.70       101
   low_risk       1.00      0.86      0.82      0.92      0.84      0.71     17104

avg / total       0.99      0.86      0.82      0.92      0.84      0.71     17205

